In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
gc.collect()

170

In [3]:
import json
from datasets import Dataset
from trl import SFTConfig, SFTTrainer
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    DataCollatorForLanguageModeling, 
    EarlyStoppingCallback
)
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
model_name = "HuggingFaceTB/SmolLM-360M"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto").to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [5]:
testing_prompt = "I want a car under $25K but don’t know what brand"
inputs = tokenizer(testing_prompt, return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )


In [6]:
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
decoded_output

'I want a car under $25K but don’t know what brand or model to look for.\n\nAlice: Well, first, you need to decide what kind of car you want. Do you prefer a sports car, a sedan, or maybe a truck? Each type has its own features and price range.\n\nBob: Hmm, I think I want a sports car. But I’m not sure which one.\n\nAlice: That’s okay! You can start by looking at the car’s specifications, like the engine size,'